### Pre Process

In [1]:
%reset -f
import pandas as pd
from collections import deque
import numpy as np

c1 = [6,1,5,7,6,2,2]
c2 = [1,5,7,6,2,8,4]
 
data_array = np.vstack((c1,c2)).T
del c1 ,c2

Adj_List = [[] for i in  range(data_array.max()+1)]#無向圖  
Adj_DList = [[] for i in  range(data_array.max()+1)]#有向圖
Adj_RDList = [[] for i in  range(data_array.max()+1)]#反向圖

#Prepare Graph data
for i,j in data_array:
    Adj_List[i].append(j)
    Adj_List[j].append(i)
    Adj_DList[i].append(j)
    Adj_RDList[j].append(i)

In [6]:
Adj_List

[[], [6, 5], [6, 8, 4], [], [2], [1, 7], [1, 7, 2], [5, 6], [2]]

In [5]:
Adj_DList

[[], [5], [8, 4], [], [], [7], [1, 2], [6], []]

In [7]:
Adj_RDList

[[], [6], [6], [], [2], [1], [7], [5], [2]]

### Initial

In [2]:
t=0
top=0
visit=np.zeros(data_array.max()+1,'i4')
visit_point=np.zeros(data_array.max()+1,'i4')

instack=np.zeros(data_array.max()+1,'i4')#是否在堆裡(boolean)
low=np.zeros(data_array.max()+1,'i4') #對index來講,最早能回朔到的源頭的in time
contract=np.zeros(data_array.max()+1,'i4')#final:儲存代表號的array
stack=np.zeros(data_array.max()+1,'i4')#存堆(tmp)

### Strongly Connected Component

In [3]:
#只有有向圖會考慮強連通分量(SCC)，無向圖整個都是強連通分量(SCC)。    
def DFS_list_SCC(i,Adj_Dlist):
    if (visit[i]!=0):return;
    global t ; global top
    visit_point[t]=i
    t+=1
    visit[i] = low[i] =t
    
    #堆一次就+1
    stack[top] = i; instack[i] = True
    print('stack:[top]',i)
    top+=1
  
    for j in Adj_Dlist[i]:
        if visit[j]==0:
            DFS_list_SCC(j,Adj_Dlist)
        #如果j有在堆裡,把i的最低low改成i,j中最小的
        #Hint:如果成環, 必會指回環的開頭
        #把成員的low[i]改成組長的,
        #這樣成員就不會在if (visit[i] == low[i]):這段去尋找強連通 
        if instack[j]==True: 
            low[i] = min(low[i], low[j])
            print('instack cond',i)
            print('Now low[i]:',low[i],' (time)')
            print('----------')
    
    #當visit[i]跟low[i]一樣,把i當頭,尋找i以下的強連通        
    if (visit[i] == low[i]): 
        print('if (visit[i] == low[i]) COND','i:',i,'/visit[i]:',visit[i])
        j=0
        #把在堆裡的從最後往後拔
        while True:
            print("Now top:",top)
            top-=1
            print("Now top-1:",top)
            print("Now stack:",stack)
            j = stack[top]
            instack[j] = False
            contract[j] = i
            if i==j:
                print('---------')
                break
    else:
        print('End i:',i)
        
DFS_list_SCC(6,Adj_DList) #Strongly Connected Component

stack:[top] 6
stack:[top] 1
stack:[top] 5
stack:[top] 7
instack cond 7
Now low[i]: 1  (time)
----------
End i: 7
instack cond 5
Now low[i]: 1  (time)
----------
End i: 5
instack cond 1
Now low[i]: 1  (time)
----------
End i: 1
instack cond 6
Now low[i]: 1  (time)
----------
stack:[top] 2
stack:[top] 8
if (visit[i] == low[i]) COND i: 8 /visit[i]: 6
Now top: 6
Now top-1: 5
Now stack: [6 1 5 7 2 8 0 0 0]
---------
stack:[top] 4
if (visit[i] == low[i]) COND i: 4 /visit[i]: 7
Now top: 6
Now top-1: 5
Now stack: [6 1 5 7 2 4 0 0 0]
---------
if (visit[i] == low[i]) COND i: 2 /visit[i]: 5
Now top: 5
Now top-1: 4
Now stack: [6 1 5 7 2 4 0 0 0]
---------
if (visit[i] == low[i]) COND i: 6 /visit[i]: 1
Now top: 4
Now top-1: 3
Now stack: [6 1 5 7 2 4 0 0 0]
Now top: 3
Now top-1: 2
Now stack: [6 1 5 7 2 4 0 0 0]
Now top: 2
Now top-1: 1
Now stack: [6 1 5 7 2 4 0 0 0]
Now top: 1
Now top-1: 0
Now stack: [6 1 5 7 2 4 0 0 0]
---------


In [4]:
contract

array([0, 6, 2, 0, 4, 6, 6, 6, 8])